<a href="https://colab.research.google.com/github/Edriczz/Formal_logic/blob/main/Final_Exam_Formal_Logic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
%pip install z3-solver

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.5/29.5 MB 79.2 MB/s eta 0:00:00


## Number 1

In [6]:
from z3 import *

# This script solves both parts of Question 1 for the Formal Logic exam.
# Part (i): Counts all possible paths for a robot under Module (a).
# Part (ii): Proves that no valid path can pass through coordinate C(2,1).

# --- Configuration ---
PATH_LEN = 10
START_A = (0, 0)
END_B = (6, 4)
POINT_C = (2, 1)
RED_COORDS = [(2,2),(3, 1), (4, 1), (2, 3), (3, 3), (4, 2)]

# ==============================================================================
#  PART 1(i): Find the number of all possible robot's movements
# ==============================================================================
print("--- Solving Question 1(i): Counting all possible paths ---")

# 1. Initialize the SMT solver for path counting
solver_count = Solver()

# 2. Define integer variables for the path
x = [Int(f'x_{i}') for i in range(PATH_LEN + 1)]
y = [Int(f'y_{i}') for i in range(PATH_LEN + 1)]

# 3. Add core constraints for Module (a)
# Start and end points
solver_count.add(x[0] == START_A[0], y[0] == START_A[1])
solver_count.add(x[PATH_LEN] == END_B[0], y[PATH_LEN] == END_B[1])

# Movement constraint (one step right or up)
for i in range(PATH_LEN):
    step_right = And(x[i+1] == x[i] + 1, y[i+1] == y[i])
    step_up = And(x[i+1] == x[i], y[i+1] == y[i] + 1)
    solver_count.add(Or(step_right, step_up))

# Forbidden (red) coordinates constraint
for i in range(1, PATH_LEN + 1):
    for r_x, r_y in RED_COORDS:
        solver_count.add(Not(And(x[i] == r_x, y[i] == r_y)))

# 4. Count all valid models (paths)
path_count = 0
while solver_count.check() == sat:
    path_count += 1
    m = solver_count.model()
    # Block the found path to find a new one in the next iteration
    block_path = Or([x[i] != m.eval(x[i]) for i in range(PATH_LEN + 1)] +
                    [y[i] != m.eval(y[i]) for i in range(PATH_LEN + 1)])
    solver_count.add(block_path)

print(f"Result: The total number of possible paths is {path_count}.\n")


# ==============================================================================
#  PART 1(ii): Show that there is no robot movement that passes coordinate C
# ==============================================================================
print("--- Solving Question 1(ii): Proof by contradiction ---")

# 1. Initialize a new, clean solver for the proof
solver_proof = Solver()

# 2. Add the same core constraints as before
# We redefine the variables for the new solver
x_p = [Int(f'xp_{i}') for i in range(PATH_LEN + 1)]
y_p = [Int(f'yp_{i}') for i in range(PATH_LEN + 1)]

solver_proof.add(x_p[0] == START_A[0], y_p[0] == START_A[1])
solver_proof.add(x_p[PATH_LEN] == END_B[0], y_p[PATH_LEN] == END_B[1])

for i in range(PATH_LEN):
    step_right = And(x_p[i+1] == x_p[i] + 1, y_p[i+1] == y_p[i])
    step_up = And(x_p[i+1] == x_p[i], y_p[i+1] == y_p[i] + 1)
    solver_proof.add(Or(step_right, step_up))

for i in range(1, PATH_LEN + 1):
    for r_x, r_y in RED_COORDS:
        solver_proof.add(Not(And(x_p[i] == r_x, y_p[i] == r_y)))

# 3. Add the specific constraint for the proof: the path MUST pass through C
path_must_pass_C = Or([And(x_p[i] == POINT_C[0], y_p[i] == POINT_C[1]) for i in range(1, PATH_LEN)])
solver_proof.add(path_must_pass_C)

# 4. Check for satisfiability
result = solver_proof.check()
print(f"Result of checking for a path that passes through C: {result}")

if result == unsat:
    print("Proof: The solver returned 'unsat', which proves by contradiction that no valid path can pass through coordinate C.")
else:
    print("Proof failed: A path through C was found, which indicates an error in the constraints.")



--- Solving Question 1(i): Counting all possible paths ---
Result: The total number of possible paths is 10.

--- Solving Question 1(ii): Proof by contradiction ---
Result of checking for a path that passes through C: unsat
Proof: The solver returned 'unsat', which proves by contradiction that no valid path can pass through coordinate C.


##Number 2

In [4]:
from z3 import *

# This script solves all three parts of Question 2 for the Formal Logic exam.
# Part (i): Counts all possible paths for a robot under Module (b).
# Part (ii): Finds the number of minimal paths by first discovering the minimal length.
# Part (iii): Proves that all minimal paths must pass through coordinates C, D, and E.

# --- Configuration ---
START_A = (0, 0)
END_B = (6, 4)
POINT_C = (2, 2)
POINT_D = (3, 2)
POINT_E = (4, 3)
RED_COORDS = [(2, 1), (3, 4), (4, 1), (2, 3), (3, 3), (4, 2)]
# The longest possible path has 10 steps (no diagonals)
MAX_POSSIBLE_LEN = 10

def add_module_b_constraints(solver, path_len):
    """A helper function to add the core constraints for a given path length."""
    x = [Int(f'x_{path_len}_{i}') for i in range(path_len + 1)]
    y = [Int(f'y_{path_len}_{i}') for i in range(path_len + 1)]

    # Start and end points
    solver.add(x[0] == START_A[0], y[0] == START_A[1])
    solver.add(x[path_len] == END_B[0], y[path_len] == END_B[1])

    # Movement constraint (right, up, or diagonal)
    for i in range(path_len):
        step_right = And(x[i+1] == x[i] + 1, y[i+1] == y[i])
        step_up = And(x[i+1] == x[i], y[i+1] == y[i] + 1)
        step_diag = And(x[i+1] == x[i] + 1, y[i+1] == y[i] + 1)
        solver.add(Or(step_right, step_up, step_diag))

    # Forbidden coordinates
    for i in range(1, path_len + 1):
        for r_x, r_y in RED_COORDS:
            solver.add(Not(And(x[i] == r_x, y[i] == r_y)))

    return x, y

def count_models(solver, x, y):
    """A helper function to count all satisfying models."""
    count = 0
    while solver.check() == sat:
        count += 1
        m = solver.model()
        block = Or([x[i] != m.eval(x[i]) for i in range(len(x))] +
                   [y[i] != m.eval(y[i]) for i in range(len(y))])
        solver.add(block)
    return count

# ==============================================================================
#  PART 2(ii): Find the number of all possible MINIMAL movements
# ==============================================================================
print("--- Solving Question 2(ii): Finding and counting minimal paths ---")
minimal_len = -1
# Find the minimal length by checking from length 1 upwards
for length in range(1, MAX_POSSIBLE_LEN + 1):
    solver_find_min = Solver()
    x, y = add_module_b_constraints(solver_find_min, length)
    if solver_find_min.check() == sat:
        minimal_len = length
        print(f"Found minimal path length to be: {minimal_len}")
        break

# Now, count the paths for the minimal length
solver_min_count = Solver()
x_min, y_min = add_module_b_constraints(solver_min_count, minimal_len)
num_minimal_paths = count_models(solver_min_count, x_min, y_min)
print(f"Result: The total number of minimal paths is {num_minimal_paths}.\n")

# ==============================================================================
#  PART 2(i): Find the number of ALL possible robot's movements
# ==============================================================================
print("--- Solving Question 2(i): Counting all possible paths ---")
total_paths = 0
# Loop from the minimal length up to the maximum possible length
for length in range(minimal_len, MAX_POSSIBLE_LEN + 1):
    solver_len = Solver()
    x_len, y_len = add_module_b_constraints(solver_len, length)
    num_paths = count_models(solver_len, x_len, y_len)
    print(f"Found {num_paths} paths of length {length}")
    total_paths += num_paths

print(f"Result: The total number of all possible paths is {total_paths}.\n")

# ==============================================================================
#  PART 2(iii): Show all minimal movements must pass C, D, and E
# ==============================================================================
print("--- Solving Question 2(iii): Proof by contradiction ---")
solver_proof = Solver()
x_p, y_p = add_module_b_constraints(solver_proof, minimal_len)

# Helper constraints to check if a path passes through a point
path_passes_C = Or([And(x_p[i] == POINT_C[0], y_p[i] == POINT_C[1]) for i in range(1, minimal_len)])
path_passes_D = Or([And(x_p[i] == POINT_D[0], y_p[i] == POINT_D[1]) for i in range(1, minimal_len)])
path_passes_E = Or([And(x_p[i] == POINT_E[0], y_p[i] == POINT_E[1]) for i in range(1, minimal_len)])

# Add the contradiction: A path exists that AVOIDS C OR AVOIDS D OR AVOIDS E
solver_proof.add(Or(Not(path_passes_C), Not(path_passes_D), Not(path_passes_E)))

result = solver_proof.check()
print(f"Result of checking for a minimal path that avoids C, D, or E: {result}")
if result == unsat:
    print("Proof: The solver returned 'unsat', proving that all minimal paths must pass through C, D, and E.")
else:
    print("Proof failed: A counterexample path was found.")


--- Solving Question 2(ii): Finding and counting minimal paths ---
Found minimal path length to be: 6
Result: The total number of minimal paths is 2.

--- Solving Question 2(i): Counting all possible paths ---
Found 2 paths of length 6
Found 11 paths of length 7
Found 24 paths of length 8
Found 25 paths of length 9
Found 10 paths of length 10
Result: The total number of all possible paths is 72.

--- Solving Question 2(iii): Proof by contradiction ---
Result of checking for a minimal path that avoids C, D, or E: unsat
Proof: The solver returned 'unsat', proving that all minimal paths must pass through C, D, and E.
